In [2]:
import serial
import time

# You will have to change this to whatever COM port the pico is assigned when
# you plug it in.
# On Windows you can open device manager and look at the 'Ports (COM & LPT)' dropdown
# the pico will show up as 'USB Serial Device'
PICO_PORT = 'COM6'

MHZ = 1000000

# helper for sending serial commands
# expects a string with the command
# if echo is set to false, that means not to worry about the response from the
# pico it signifigantly reduces communication time when sending many
# instructions, but you lose out on the debugging info from the pico
def send(command: str, echo = True) -> str:
    # pico is expecting a newline to end every command
    if command[-1] != '\n':
        command += '\n'

    resp = ''
    try:
        conn = serial.Serial(PICO_PORT, baudrate = 152000, timeout = 0.1)
        conn.write(command.encode())
        if echo:
            resp = conn.readlines()
            resp = "".join([s.decode() for s in resp])

    except Exception as e:
        print("Encountered Error: ", e)

    finally:
        conn.close()

    return resp



## Single Stepping Freq Sweep

In [21]:
s0 = 53 * MHZ
e0 = 59 * MHZ

t = 5000

dt = 1000
pow = 0
asf = 1

send(f"""mode 0 1
setchannels 1
"""
)

dt = 500
steps = t / dt
df = (e0 - s0) / steps
for i in range(int(steps)):
    send(f'set 0 {i} {s0 + i * df} {asf} {pow} {dt}', echo=False)
    
send(f'set 0 {i+1} {s0} {asf} {pow} {dt}', echo=False)
send(f'set 4 {i + 2}')
send('start')
    
steps

10.0

### Same thing but as a sweep

In [22]:
sd = (e0 - s0) / (t)

send(f"""mode 2 1
setchannels 1
set 0 0 {s0} {e0} {sd} {1} {t}
set 0 1 {s0} {s0} 0 0 {dt}
set 4 2
start
"""
)

'ok\r\nok\r\nSet ins #0 for channel 0 from 52999999.956228 Hz to 59000000.008382 Hz with delta 1200.009137 Hz and rate of 1\r\nok\r\nSet ins #1 for channel 0 from 52999999.956228 Hz to 52999999.956228 Hz with delta 0.000000 Hz and rate of 0\r\nok\r\nok\r\nok\r\n'

## Looking closely at downward sweeps

In [18]:
f53 = 53000000
f56 = 56000000
f59 = 59000000

t = 0

print(send(
f"""mode 2 1
setchannels 1
set 0 0 {f53} {f56} {8000} 1 {t}
set 0 1 {f56} {f53} {2000} 1 {t}
set 4 2
start
"""
))

ok
ok
Set ins #0 for channel 0 from 52999999.956228 Hz to 55999999.982305 Hz with delta 7999.944501 Hz and rate of 1
ok
Set ins #1 for channel 0 from 55999999.982305 Hz to 52999999.956228 Hz with delta 2000.015229 Hz and rate of 1
ok
ok
ok



## Sawish Tooth

In [143]:
sweepRange = 60 * MHZ / 16
center = 83.62 * MHZ
startPoint = center - sweepRange / 2
endPoint = center + sweepRange / 2

dwellTime = 100e-6
cycleTime = 1 / (125 * MHZ)
dwellCycles = dwellTime / cycleTime

print(send(f"""abort
mode 0 1
setfreq 0 {startPoint}
setfreq 1 {startPoint}
setfreq 2 {startPoint}
setfreq 3 {startPoint}
mode 2 1
setchannels 1
"""))

i = 0
for decade in range(1, 5):
    if decade == 1:
        div = 20
    else:
        div = 1
    for l in [6, 3, 1]:
        sweepTime = l * 10**(-decade)
        sweepCycles = sweepTime / cycleTime
        delta = sweepRange / sweepCycles * div
        send(f'set 0 {i} {startPoint} {endPoint} {delta} {div} {sweepCycles}', echo=False)
        send(f'set 0 {i + 1} {startPoint} {startPoint} 0 {1} {dwellCycles}', echo=False)
        i += 2
        
send(f'set 4 {i}', echo=False)
print('Ready')
        

ok
ok
set freq: 81744999.974035
ok
set freq: 81744999.974035
ok
set freq: 81744999.974035
ok
set freq: 81744999.974035
ok
ok
ok

1.0
2.0
6.0
0.5000000000000001
1.0000000000000002
3.0
5.0
10.0
30.0
50.0
100.0
300.0
Ready


In [144]:
send('start')

'ok\r\n'